Step 1: Data Loading

Reads the dataset from the .csv Excel sheets.

In [15]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from imblearn.over_sampling import SMOTE
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns

# Enable inline plotting
%matplotlib inline

# Step 1: Load Data
file_path = '/workspaces/codespaces-jupyter/data/data.xlsx'  
xls = pd.ExcelFile(file_path)

# Display sheet names for verification
print("Sheet Names:", xls.sheet_names)

# Load each sheet into a DataFrame
sales_df = pd.read_excel(xls, sheet_name='sales')
product_df = pd.read_excel(xls, sheet_name='product')
demand_df = pd.read_excel(xls, sheet_name='demand')
inci_category_df = pd.read_excel(xls, sheet_name='inci_category')

print("\nData Loaded Successfully!")

Sheet Names: ['sales', 'product', 'demand', 'inci_category']

Data Loaded Successfully!


Step 2: Data Inspection

In [16]:
# Step 2: Inspecting the Data
print("Sales Data Head:")
display(sales_df.head())

print("Product Data Head:")
display(product_df.head())

print("Demand Data Head:")
display(demand_df.head())

print("INCI Category Data Head:")
display(inci_category_df.head())

Sales Data Head:


,Customer,SP,Sales 2024
0,Customer EMEA,30164505,10.00
1,Customer NA,30527725,0.03
2,Customer Asia,30538226,0.06
3,Customer SA,30746855,0.06
4,Customer SA,30222192,0.10


Product Data Head:


,INCI,SP,Sales_Product,Composition
0,propylene glycol,30061790,"1,2-PROPYLENE GLYCOL CARE",Propylene Glycol
1,butylene glycol,30459417,A00067 BETAPUR,Butylene Glycol (and) Pentylene Glycol (and) P...
2,pentylene glycol,30459417,A00067 BETAPUR,Butylene Glycol (and) Pentylene Glycol (and) P...
3,peumus boldus leaf extract,30459417,A00067 BETAPUR,Butylene Glycol (and) Pentylene Glycol (and) P...
4,xanthan gum,30459417,A00067 BETAPUR,Butylene Glycol (and) Pentylene Glycol (and) P...


Demand Data Head:


,INCI,Demand 2025,Customer,number of beauty care products launched
0,"1,2-hexanediol",NaN,Customer Asia,859
1,"1,2-hexanediol",NaN,Customer EMEA,27
2,"1,2-hexanediol",NaN,Customer SA,3
3,"1,2-hexanediol",NaN,Customer NA,318
4,"1,4-butanediol",NaN,Customer Asia,1


INCI Category Data Head:


,INCI Name,Category
0,hydrated silica,Thickener
1,oryza sativa starch,Thickener
2,peumus boldus leaf extract,Actives
3,peucedanum graveolens extract,Actives
4,zea mays kernel extract,Actives


Step 3: Data Cleaning

Removes duplicates.
Handles missing values.
Ensures consistency across datasets.

In [17]:
# Step 3: Data Cleaning

# Clean Data
sales_df = sales_df.drop_duplicates().fillna(0)
product_df = product_df.drop_duplicates().fillna("Unknown")
demand_df = demand_df.drop_duplicates()
demand_df['Demand 2025'] = demand_df['Demand 2025'].fillna(demand_df['Demand 2025'].mean())
inci_category_df = inci_category_df.drop_duplicates()
inci_category_df['Category'] = inci_category_df['Category'].fillna("Other")

# Cleaning Tables
for name, df in zip(['Sales', 'Product', 'Demand', 'INCI Category'], [sales_df, product_df, demand_df, inci_category_df]):
    df.drop_duplicates(inplace=True)
    print(f"{name} Table Cleaned: {df.shape}")

Sales Table Cleaned: (496, 3)
Product Table Cleaned: (1260, 4)
Demand Table Cleaned: (4212, 4)
INCI Category Table Cleaned: (511, 2)


Step 4: Impute Data

In [18]:
# Impute Missing Values
sales_df.fillna(0, inplace=True)
product_df.fillna("Unknown", inplace=True)
demand_df['Demand 2025'] = demand_df['Demand 2025'].fillna(demand_df['Demand 2025'].mean())
inci_category_df['Category'] = inci_category_df['Category'].fillna("Other")

# Step 4: Sanitize Column Names
# Clean and sanitize individual tables
def clean_and_sanitize(df):
    df.drop_duplicates(inplace=True)
    df.columns = (
        df.columns.astype(str)
        .str.replace(r'[\[\]<>,]', '', regex=True)
        .str.strip()
        .str.replace(' ', '_')
    )
    return df

sales_df = clean_and_sanitize(sales_df)
product_df = clean_and_sanitize(product_df)
demand_df = clean_and_sanitize(demand_df)
inci_category_df = clean_and_sanitize(inci_category_df)

# Impute missing values
sales_df = sales_df.fillna(0)  # Reassign filled DataFrame
product_df = product_df.fillna("Unknown")  # Reassign filled DataFrame

# Ensure DataFrame copies to avoid chained assignment issues
demand_df = demand_df.copy()
inci_category_df = inci_category_df.copy()

# Handle missing values explicitly
demand_df['Demand_2025'] = demand_df['Demand_2025'].fillna(demand_df['Demand_2025'].mean())
inci_category_df['Category'] = inci_category_df['Category'].fillna("Other")

print("Missing values handled without warnings.")

# Step 5: Merge Tables
# Impute missing values
sales_df.fillna(0, inplace=True)
product_df.fillna("Unknown", inplace=True)
demand_df['Demand_2025'].fillna(demand_df['Demand_2025'].mean(), inplace=True)
inci_category_df['Category'] = inci_category_df['Category'].fillna("Other")

# Merge datasets
merged_df = pd.merge(sales_df, product_df, on="SP", how="left")
merged_df = pd.merge(merged_df, demand_df, on=["Customer", "INCI"], how="left")
merged_df = pd.merge(merged_df, inci_category_df, left_on="INCI", right_on="INCI_Name", how="left")
merged_df.drop(columns=["INCI_Name"], inplace=True)

print("\nMerged DataFrame Shape:", merged_df.shape)
display(merged_df.head())

Missing values handled without warnings.

Merged DataFrame Shape: (810, 9)


/tmp/ipykernel_3193/3436487576.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  demand_df['Demand_2025'].fillna(demand_df['Demand_2025'].mean(), inplace=True)


,Customer,SP,Sales_2024,INCI,Sales_Product,Composition,Demand_2025,number_of_beauty_care_products_launched,Category
0,Customer EMEA,30164505,10.00,polaxamer 407,PLURACARE F127 NF PRILL SURF.,Poloxamer 407,NaN,NaN,NaN
1,Customer EMEA,30164505,10.00,poloxamer 407,PLURACARE F127 NF PRILL SURF.,Poloxamer 407,2960.106825,1.0,Surfactant
2,Customer EMEA,30164505,10.00,poloxamer 407,PLURACARE F127 NF PRILL SURFACTANT,Poloxamer 407,2960.106825,1.0,Surfactant
3,Customer NA,30527725,0.03,hydralyzed wheat protein,GLUADIN WP,Hydrolyzed Wheat Protein,NaN,NaN,Hair Conditioner
4,Customer NA,30527725,0.03,hydrolyzed wheat protein,GLUADIN WP,Hydrolyzed Wheat Protein,928.000000,81.0,Hair Conditioner


Step 6: Outlier detection

In [19]:
# Step 6: Outlier Detection and Handling

# Identify outliers using Z-score
numeric_cols = merged_df.select_dtypes(include=np.number).columns
z_scores = merged_df[numeric_cols].apply(zscore)
outliers = (np.abs(z_scores) > 3).sum()
print(f"Outliers Detected:\n{outliers}")

# Cap extreme outliers
for col in numeric_cols:
    upper_limit = merged_df[col].mean() + 3 * merged_df[col].std()
    lower_limit = merged_df[col].mean() - 3 * merged_df[col].std()
    merged_df[col] = np.where(merged_df[col] > upper_limit, upper_limit, merged_df[col])
    merged_df[col] = np.where(merged_df[col] < lower_limit, lower_limit, merged_df[col])

Outliers Detected:
SP                                         0
Sales_2024                                 5
Demand_2025                                0
number_of_beauty_care_products_launched    0
dtype: int64


Step 7: Feature Engineering

Adding Ratios to Understand Relationships
    Why?

    Individual sales and demand figures provide standalone information, but a ratio between them helps the model understand whether sales are meeting demand effectively.

    What We Did:
    We created the feature Sales_Demand_Ratio:

    Sales_Demand_Ratio= Sales_2024 / (Demand_2025+1)
    The +1 prevents division by zero.

    Impact:
    This feature helps capture efficiency or underperformance in meeting demand, a key driver for forecasting future trends.

Aggregating Category-Level Information

    Why?
    Products belonging to the same category often share trends (e.g., luxury products might sell more in specific regions).
    Adding aggregated metrics can highlight category-level behaviors.

    What We Did:
    We computed Category_Sales_Ratio:

    Category_Sales_Ratio= Total_Sales_in_Category / Sales_2024
​	
 
    This feature shows how a specific product contributes to overall category sales.
    Impact:
    It helps the model identify outlier products or regions where a category dominates sales.

In [20]:
# Step 7: Feature Engineering
# Add interaction features
merged_df['Sales_Demand_Ratio'] = merged_df['Sales_2024'] / (merged_df['Demand_2025'] + 1)
merged_df['Category_Sales_Ratio'] = merged_df.groupby('Category')['Sales_2024'].transform('sum') / merged_df['Sales_2024']

# Logarithmic transformation for normalization
merged_df['Log_Sales_2024'] = np.log1p(merged_df['Sales_2024'])

print("\nFeature Engineering Completed.")
display(merged_df.head())


Feature Engineering Completed.


,Customer,SP,Sales_2024,INCI,Sales_Product,Composition,Demand_2025,number_of_beauty_care_products_launched,Category,Sales_Demand_Ratio,Category_Sales_Ratio,Log_Sales_2024
0,Customer EMEA,30164505.0,10.00,polaxamer 407,PLURACARE F127 NF PRILL SURF.,Poloxamer 407,NaN,NaN,NaN,NaN,NaN,2.397895
1,Customer EMEA,30164505.0,10.00,poloxamer 407,PLURACARE F127 NF PRILL SURF.,Poloxamer 407,2960.106825,1.0,Surfactant,0.003377,85325.915587,2.397895
2,Customer EMEA,30164505.0,10.00,poloxamer 407,PLURACARE F127 NF PRILL SURFACTANT,Poloxamer 407,2960.106825,1.0,Surfactant,0.003377,85325.915587,2.397895
3,Customer NA,30527725.0,0.03,hydralyzed wheat protein,GLUADIN WP,Hydrolyzed Wheat Protein,NaN,NaN,Hair Conditioner,NaN,851931.299807,0.029559
4,Customer NA,30527725.0,0.03,hydrolyzed wheat protein,GLUADIN WP,Hydrolyzed Wheat Protein,928.000000,81.0,Hair Conditioner,0.000032,851931.299807,0.029559


Step 8: Modeling

In [21]:
# Step 8: Advanced Modeling Approaches
# Prepare features and target
X = merged_df.drop(columns=['Demand_2025'])
y = merged_df['Demand_2025']

# Check for missing values in y
print("Missing Values in Target (y):")
print(y.isnull().sum())

# Handle missing values in y
if y.isnull().sum() > 0:
    # Option 1: Remove rows with missing y
    valid_indices = ~y.isnull()
    X = X[valid_indices]
    y = y[valid_indices]
    print("Rows with missing y removed.")

    # Option 2: Impute missing values
    # Uncomment the following lines if imputation is preferred
    # y = y.fillna(y.mean())  # Replace NaNs with mean (for regression)
    # y = y.fillna(y.mode()[0])  # Replace NaNs with mode (for classification)

# Verify no missing values remain
print("Missing Values in Target (y) After Handling:")
print(y.isnull().sum())

# Encode categorical variables
X = pd.get_dummies(X, drop_first=True)
imputer = SimpleImputer(strategy='mean')  # Replace with 'median', 'most_frequent', or 'constant' if needed
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Ensure all feature names are strings
X.columns = X.columns.astype(str)

# Remove invalid characters and sanitize feature names
X.columns = (
    X.columns
    .str.replace(r'[\[\]<>,]', '', regex=True)  # Remove [, ], <, >, etc.
    .str.strip()  # Remove leading/trailing spaces
    .str.replace(' ', '_')  # Replace spaces with underscores
)

# Verify cleaned column names
print("Cleaned Feature Names:", list(X.columns))

# Validate feature names
assert all(isinstance(col, str) for col in X.columns), "Feature names must be strings!"


# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train models
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, random_state=42)
}

results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results.append({
        "Model": name,
        "MAE": mean_absolute_error(y_test, y_pred),
        "MSE": mean_squared_error(y_test, y_pred),
        "R2": r2_score(y_test, y_pred)
    })

results_df = pd.DataFrame(results)
print("\n### Model Performance")
display(results_df)

# Save final data
print("\n### Save Final Data")
merged_df.to_csv("cleaned_prepared_table.csv", index=False)
print("Data saved to 'cleaned_prepared_table.csv'")

Missing Values in Target (y):
173
Rows with missing y removed.
Missing Values in Target (y) After Handling:
0
Cleaned Feature Names: ['SP', 'Sales_2024', 'number_of_beauty_care_products_launched', 'Sales_Demand_Ratio', 'Category_Sales_Ratio', 'Log_Sales_2024', 'Customer_Customer_EMEA', 'Customer_Customer_NA', 'Customer_Customer_SA', 'INCI_acrylate_copolymer', 'INCI_acrylates_copolymer', 'INCI_acrylates/beheneth-25_methacrylate_copolymer', 'INCI_acrylates/c12-22_alkyl_methacrylate_copolymer', 'INCI_algin', 'INCI_alpinia_galanga_extract', 'INCI_ammonium_laureth_sulfate', 'INCI_ammonium_lauryl_sulfate', 'INCI_arachidyl_propionate', 'INCI_ascorbyl_palmitate', 'INCI_beheneth-25', 'INCI_behentrimonium_chloride', 'INCI_behenyl_alcohol', 'INCI_benzophenone-4', 'INCI_beta-glucan', 'INCI_bis-ethylhexyloxyphenol_methoxyphenyl_triazine', 'INCI_buteth-3', 'INCI_butylene_glycol', 'INCI_butyrospermum_parkii_butter', 'INCI_c12-15_alkyl_benzoate', 'INCI_c12-18_alkanoyl_glycerin/sebacic_acid_copolymer',

,Model,MAE,MSE,R2
0,Linear Regression,1.509398e+09,1.642514e+19,-5.732211e+10
1,Random Forest,1.832946e+03,5.183952e+07,8.190852e-01
2,XGBoost,1.591820e+03,8.809852e+07,6.925450e-01



### Save Final Data
Data saved to 'cleaned_prepared_table.csv'
